# 抽出

抽出チェーンはドキュメントからエンティティを抽出するためのスキーマを指定するためにOpenAIの `functions` パラメータを使用します。これにより、モデルが正確に私たちが望むエンティティやプロパティのスキーマを、適切なタイプで出力することができるようになります。

抽出チェーンは、同じ文章から複数のエンティティとそのプロパティを抽出したい場合（例えば、この文章で誰が言及されていたか？）に使用されます。

In [ ]:
!pip install langchainplus_sdk

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain, create_extraction_chain_pydantic
from langchain.prompts import ChatPromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

## エンティティの抽出

エンティティを抽出するためには、以下のようなスキーマを作成する必要があります。ここでは、見つけたいすべてのプロパティとそれらが持つべきタイプを指定します。また、これらのプロパティの中でどれが必須で、どれがオプションかも指定できます。

In [ ]:
schema = {
    "properties": {
        "person_name": {"type": "string"},
        "person_height": {"type": "integer"},
        "person_sex": {"type": "string"},
        "person_penis_size": {"type": "integer"},
        "person_sex_position": {"type": "string"},
        "person_hair_color": {"type": "string"},
    },
    "required": ["person_name", "person_height"],
}

In [ ]:
inp = """
アレックスは身長が152.4cmです。クラウディアはアレックスよりも30.5cm背が高く彼よりもジャンプが高いです。クラウディアは黒髪でアレックスは金髪です。
アレックスの犬のフロスティはラブラドールでかくれんぼが好きです。
"""

In [ ]:
chain = create_extraction_chain(schema, llm)

私たちが見て取れるように、必要なエンティティとそのプロパティを必要な形式で抽出しました：

In [ ]:
chain.run(inp)

## Pydanticの例

また、Pydanticスキーマを使用して必要なプロパティや型を選択して、厳密には必要でないものを'Optional'として設定することもできます。

そこで `create_extraction_chain_pydantic` 関数を使うことで、Pydantic のスキーマを入力として送信し、希望するスキーマを反映したオブジェクトのインスタンスを出力することができます。

このように、新しいクラスや関数を Python で指定するのと同じように、純粋に Pythonic な型を使ってスキーマを指定することができます。

In [ ]:
from typing import Optional, List
from pydantic import BaseModel, Field

In [ ]:
class Properties(BaseModel):
    person_name: str
    person_height: float
    person_hair_color: str
    person_sex: Optional[str]
    person_sex_position: Optional[str]
    person_penis_length: Optional[float]

In [ ]:
chain = create_extraction_chain_pydantic(pydantic_schema=Properties, llm=llm)

In [ ]:
inp = """
アレックスは身長が152.4cmです。クラウディアはアレックスよりも30.5cm背が高く彼よりもジャンプが高いです。クラウディアは黒髪でアレックスは金髪です。
アレックスの犬のフロスティはラブラドールでかくれんぼが好きです。
"""

私たちが見て取れるように、必要なエンティティとそのプロパティを必要な形式で抽出しました：

In [ ]:
chain.run(inp)